### Import mix module 

In [1]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
from Optimiser.config import get_config, normalize_action, denormalize_action
import argparse

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.9985486850747627

In [ ]:
denormalize_action(np.array([-3.604144,0.5862619,-1.3705522,-1.853153]))

In [9]:
def haaqi_reward_fn(solution: np.ndarray, is_normalised = True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
    print(f'solution:{solution}')
    thre, ratio, attack, release = solution
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return round(1-score,3)

In [ ]:
###inital
haaqi_reward_fn(np.array([-30. ,   8.1,   1. , 100.]),False)

In [ ]:
import numpy as np
m1=np.array([-0.5,-0.5,-0.5,-0.5])
m2=np.array([0.5,0.5,0.5,0.5])

def f(x):
    return -np.sum(np.log(np.sum((x-m1)**2)+0.00001)-np.log(np.sum((x-m2)**2)+0.01))

## Ranking

### function

In [ ]:
######ranking corresspoding funciton 
def haaqi_reward_fn(solution: np.ndarray) -> float:
    thre = solution[0]
    thre = thre - 15
    ratio = solution[1]
    ratio = ratio + 5
    attack = solution[2]
    attack = attack + 10
    release = solution[3] 
    release = release + 300
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return 1-score

### GA

In [ ]:
###Worst result for GA Until the 11.July
###haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))
####Worst result for GA Until the 12.July 
haaqi_reward_fn(np.array([-14.9, 3.3, -9.0, -193]))

### RL in Continous

In [ ]:
####Worst result for GA Until the 12.July [-1.2019364  -5.0801964  -3.631278  -54.986313]
haaqi_reward_fn(np.array([-1.2019,-5.0801,-3.6313,-54.9863]))

### File Clear

In [ ]:
Noise_Generator_MP3.EraseTheMp3Mixing()

### Import Discrete RL module

In [ ]:
import Optimiser

In [ ]:
import Optimiser.discrete_RL_train
from Optimiser.discrete_RL_train import discrete_RL_train as RLTrain

In [ ]:
trainner = RLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [ ]:
###Worst result for discrete ReinforcementL Until the 11.July
haaqi_reward_fn(np.array([ -3.9 ,  -0.5 ,  -4.4 , -52.72]))

### Import Continous RL

In [ ]:
import Optimiser

In [ ]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

In [ ]:
trainner = CRLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [ ]:
RL_data, RL_F_data = trainner.save_results(filefold=Mixing_Path, para_columns=['Thre','Ratio','Attk','Release'],is_outputfulldata = False)

In [ ]:
final_reward_list

In [ ]:
REINFORCE_logs

### Import GA module

In [ ]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [ ]:
GA_Opt = GeneticOptimiser()

In [ ]:
GA_Opt.ga_init_env()

In [ ]:
GA_Opt.set_fitnessfun(haaqi_reward_fn)

In [ ]:
GA_Opt.run()

In [ ]:
Evo_Data, Evo_Data_Full = GA_Opt.save_results(filefold=Mixing_Path, genre_columns=['Thre','Ratio','Attk','Release'], is_outputfulldata = True)

In [ ]:
Evo_Data

In [ ]:
Evo_Data_Full

### Import MC Optimiser

In [6]:
import Optimiser.mc_opt
from Optimiser.mc_opt import MonteCarloOptimiser

2025-07-16 17:27:40.396879: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 17:27:40.467354: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 17:27:40.467410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 17:27:40.474221: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 17:27:40.499637: I tensorflow/core/platform/cpu_feature_guar

In [10]:
mc = MonteCarloOptimiser(reward_fn = haaqi_reward_fn)
per_step_bests = mc.optimise_by_steps(n_steps=5, sampling_per_step=3)

solution:[-11.   3.   2. 107.]
solution:[ -6.   9.  13. 392.]
solution:[-14.   9.  17. 101.]
 Step 1/5 → best R = 0.0790
solution:[ -4.   1.  15. 170.]
solution:[ -4.   6.   7. 269.]
solution:[-29.   2.  14. 359.]
 Step 2/5 → best R = 0.1450
solution:[-12.   4.  20. 492.]
solution:[ -9.   7.  14. 256.]
solution:[-26.   7.  11. 224.]
 Step 3/5 → best R = 0.1710
solution:[-15.   9.  19. 243.]
solution:[-13.   4.  12. 235.]
solution:[-18.   9.   5. 349.]
 Step 4/5 → best R = 0.1420
solution:[-27.   8.  16. 196.]
solution:[ -4.   2.   7. 160.]
solution:[-16.   8.   5. 121.]
 Step 5/5 → best R = 0.1690


In [ ]:
np.round(rng.uniform([-30.0,  1.0,  1.0, 100.0], [  0.0, 10.0, 20.0, 500.0], size=4),2)